In [201]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from random import randrange
from sklearn.metrics import mean_squared_error as mse

In [202]:
def mtrx_reg(x, y, x_pred, y_pred):
    z_pred = np.array([[el[0], el[1], 1] for el in zip(x_pred, y_pred)])
    b_x = la.inv(z_pred.transpose().dot(z_pred)).dot(z_pred.transpose().dot(x))
    b_y = la.inv(z_pred.transpose().dot(z_pred)).dot(z_pred.transpose().dot(y))
    return np.array([b_x, b_y])

In [203]:
mtrx_reg([142.123,152.123,122.123,112.123, 0, 0, 0], [99.570*2,0,0,0,10,15,-10], [1680.721,2065.147,911.499,526.798,1290.920,1285.731,1300.953], [968.321,964.366,976.283,980.035, 587.055,394.727,1355.487])

array([[ 2.68487901e-02,  7.49157373e-02, -2.58930630e+01],
       [ 5.18382029e-02,  1.57937447e-03, -3.79199198e+01]])

In [204]:
def mtrx_reg_robust_lms(x, y, x_pred, y_pred, S, treshold):
    '''
    3 random points
    '''    
    res = list()
    x_gen = x
    y_gen = y
    x_pred_gen = x_pred
    y_pred_gen = y_pred
    z_pred_gen = np.array([[el[0], el[1], 1]
                           for el in zip(x_pred_gen, y_pred_gen)])

    for i in range(S):
        el1, el2, el3 = randrange(len(x_gen)), randrange(len(x_gen)), randrange(len(x_gen))
        while (el1 == el2) or (el1 == el3) or (el2 == el3):
            el1, el2, el3 = randrange(len(x_gen)), randrange(
                len(x_gen)), randrange(len(x_gen))

        z_pred = z_pred_gen[[el1, el2, el3]]

        x = np.array(x_gen)[[el1, el2, el3]]
        y = np.array(y_gen)[[el1, el2, el3]]

        x_pred = np.array(x_pred_gen)[[el1, el2, el3]]
        y_pred = np.array(y_pred_gen)[[el1, el2, el3]]

        b_x = la.inv(z_pred.transpose().dot(z_pred)).dot(
            z_pred.transpose().dot(x))
        b_y = la.inv(z_pred.transpose().dot(z_pred)).dot(
            z_pred.transpose().dot(y))

        principal_points = {'x': [], 'y': [], 'x_pred': [], 'y_pred': []}
        num_inl = 0
        # print(z_pred[0].dot(b_x))
        # break
        for i in range(len(x_gen)):
            if (x_gen[i] - z_pred_gen[i].dot(b_x))**2 + (y_gen[i] - z_pred_gen[i].dot(b_y))**2 < treshold:
                principal_points['x'].append(x_gen[i])
                principal_points['y'].append(y_gen[i])
                principal_points['x_pred'].append(x_pred_gen[i])
                principal_points['y_pred'].append(y_pred_gen[i])
                num_inl += 1

        res.append((principal_points, num_inl))
    
    max_num_inl = max(np.array(res)[:, 1])
    for el in res:
        if el[1] == max_num_inl:
            return mtrx_reg(el[0]['x'], el[0]['y'], el[0]['x_pred'], el[0]['y_pred'])

In [205]:
mtrx_reg_robust_lms([142.123,152.123,122.123,112.123, 0, 0, 0], [99.570*2,0,0,0,10,15,-10], [1680.721,2065.147,911.499,526.798,1290.920,1285.731,1300.953], [968.321,964.366,976.283,980.035, 587.055,394.727,1355.487], 4, 0.1)

array([[ 2.95035994e-02,  3.42611969e-01, -2.39218840e+02],
       [-2.65253971e-04, -2.59572788e-02,  2.55807708e+01]])